In [5]:

import seaborn as sns
import missingno as msno
from datetime import datetime
import statsmodels.api as sm
import os
import pandas as pd
import json
import numpy as np
import pickle
import ast
pd.set_option('display.max_columns', None)

# **ETL_Archivos YELP**
---

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **checkin.json**
---

**Carga de lo datos**

In [ ]:
df_checkin_yelp=pd.read_json(r'/content/drive/MyDrive/P_F_H/checkin.json',lines=True)
df_checkin_yelp.head(2)
# Archivo con datos que posiblemente no se usen

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."


**Buscamos elementos nulos**

In [ ]:
df_checkin_yelp.isnull().sum()

business_id    0
date           0
dtype: int64

**Revisamos si tiene duplicados**

In [ ]:
df_checkin_yelp.duplicated().sum()

0

**Desanidamos la fecha (en Fecha y hora)**

In [ ]:
desanidados = []

for _, row in df_checkin_yelp.iterrows():
    business_id = row['business_id']
    dates = row['date'].split(', ')
    for date in dates:
        time, date = date.split(' ')
        desanidados.append([business_id, time, date])

df_desanidado = pd.DataFrame(desanidados, columns=['business_id', 'hour', 'fecha'])

In [ ]:
df_desanidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13356875 entries, 0 to 13356874
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   business_id  object
 1   hour         object
 2   fecha        object
dtypes: object(3)
memory usage: 305.7+ MB


**Cargamos datos en tabla checking la cual se puede usar para una analisis alternativo**

In [ ]:
df_desanidado.to_csv('Checkin_Yelp.csv', index=False)

## **user.parquet**
---

**Extraccion de los datos(desde la fuente)**

In [ ]:
df_y1=pd.read_parquet('/content/drive/MyDrive/P_F_H/user.parquet')
df_y1.head(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,250,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,3.74,1145,264,184,157,251,1847,7054,3131,3131,1521,1946


**Transformacion de los datos**

**Busqueda valores nulos**

In [ ]:
df_y1.isnull().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

**Realizamos un filtro de los usuarios segun cantidad de reviews que realizaron  y si estas fueron consideradas utiles**

In [ ]:
df_y1[(df_y1['review_count']>=10) & (df_y1['useful']>=10)].head(3)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,250,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,3.74,1145,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,3.32,89,13,10,17,3,66,96,119,119,35,18


**Eliminamos duplicados**

In [ ]:
#Conteo de duplicados
df_y1.duplicated().sum()

117700

In [ ]:
df_y1.drop_duplicates(inplace=True)

**Eliminamos columnas que no se usaran**

In [ ]:
df_y1.drop(['name','compliment_hot','compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note','compliment_plain','compliment_cool','compliment_funny','compliment_writer','compliment_photos'], axis=1,inplace=True)

In [ ]:
df_y1.drop(['useful','funny','cool','fans','friends'], axis=1,inplace=True)

**Aplicamos transformaciones a los tipos de datos**

In [ ]:
#cambiamos a tipo datetime la variable yelping_since
df_y1['yelping_since'] = pd.to_datetime(df_y1['yelping_since'])

#Creamos una columna año a partir de elite que devuelva una lista de años separado por comas
df_y1['years'] = df_y1['elite'].str.split(',')

In [ ]:
#reemplazamos valores vacios por np.nan en years
df_y1['years'] = df_y1['years'].replace("", np.nan)

#reemplazamos 20 po 2020 en years
df_y1['years'] = df_y1['years'].replace("20", "2020")

In [ ]:
df_y1.head()

,user_id,review_count,yelping_since,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,585,2007-01-25 16:47:26,3.91
1,j14WgRoU_-2ZE1aw1dXrJg,4333,2009-01-25 04:35:42,3.74
2,2WnXYQFK0hXEoTxPtV2zvg,665,2008-07-25 10:41:00,3.32
3,SZDeASXq7o05mMNLshsdIA,224,2005-11-29 04:38:33,4.27
4,hA5lMy-EnncsH4JoR-hFGQ,79,2007-01-05 19:40:59,3.54


In [ ]:
df_y1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987897 entries, 0 to 1987896
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        object        
 1   review_count   int64         
 2   yelping_since  datetime64[ns]
 3   elite          object        
 4   average_stars  float64       
 5   years          object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 106.2+ MB


**Carga de datos resultante**

In [ ]:

# archivo con los usuario mas votados o reviews significativas
df_y1.to_parquet('User_Yelp.parquet', index=False, compression='gzip')

**Eliminamos las columnas elite y years del dataframe original user.parquet**

In [ ]:
df_y1.drop(columns=['elite'], inplace=True)
df_y1.drop(columns=['years'], inplace=True)

In [ ]:
df_User_Yelp=df_y1.copy()

In [ ]:
df_User_Yelp.to_csv('User_Yelp.csv', index=False)

## **business.pkl**
---

**Carga de datos**

In [ ]:
df_business= pd.read_pickle('/content/drive/MyDrive/P_F_H/business.pkl')

**Se agregan nuevas columnas al dataframe**

In [ ]:
df_business['NAME']=None
df_business['REVIEW_COUNT']=None
df_business['POSTAL_CODE']=None
df_business['CITY']=None
df_business['STATE']=None
df_business['BUSINESS_ID']=None
df_business['ADDRESS']=None
df_business['LATITUDE']=None
df_business['LONGITUDE']=None
df_business['STARS']=None
df_business['IS_OPEN']=None
df_business['ATTRIBUTES']=None
df_business['CATEGORIES']=None
df_business['HOURS']=None

In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 42 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null

**Creamos la columna "NAME" del DataFrame df_business extrayendo los datos y agregando .**



In [ ]:
for index,i in enumerate(df_business.name.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['NAME'][index]=''.join(arr)

In [ ]:
df_business.NAME

0         Abby Rappoport, LAC, CMQ
1                    The UPS Store
2                           Target
3               St Honore Pastries
4         Perkiomen Valley Brewery
                    ...           
150341                Binh's Nails
150342        Wild Birds Unlimited
150343           Claire's Boutique
150344    Cyclery & Fitness Center
150345                     Sic Ink
Name: NAME, Length: 150346, dtype: object

**Creamos la columna 'CITY' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.city.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CITY'][index]=''.join(arr)

**Creamos la columna 'STATE' del DataFrame df_business extrayendo los datos y agregando .**


In [ ]:
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['STATE'][index]=''.join(arr)

**Creamos la columna 'REVIEW_COUNT' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.review_count.values):
    arr=[]
    for e in i:
        if isinstance(e,int):
         arr.append(e)
    df_business['REVIEW_COUNT'][index]=arr[0]

**Creamos la columna 'BUSINESS_ID' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.business_id.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['BUSINESS_ID'][index]=''.join(arr)


**Creamos la columna 'ADDRESS' del DataFrame df_business extrayendo los datos y agregando .**



In [ ]:
for index,i in enumerate(df_business.address.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['ADDRESS'][index]=''.join(arr)

**Creamos la columna 'POSTAL_CODE' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.postal_code.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['POSTAL_CODE'][index]=''.join(arr)

**Creamos la columna 'LATITUDE' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.latitude.values):
    arr=[]
    for e in i:
       if e>1:
         arr.append(e)
    df_business['LATITUDE'][index]=arr[0]

**Creamos la columna 'LONGITUDE' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.longitude.values):
    arr=[]
    for e in i:
        if e<-1:
            arr.append(e)
    df_business['LONGITUDE'][index]=arr[0]

**Creamos la columna 'STARS' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.stars.values):
    arr=[]
    for e in i:
       if e>0.1:
         arr.append(e)
    df_business['STARS'][index]=arr[0]

**Creamos la columna 'IS_OPEN' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.is_open.values):
    arr=[]
    for e in i:
       if e >=0:
         arr.append(e)
    df_business['IS_OPEN'][index]=arr[0]

**Creamos la columna 'ATRIBUTES' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.attributes.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:
     df_business['ATTRIBUTES'][index]=arr[0]

**Creamos la columna 'CATEGORIES' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
# AGREGA LAS CADENAS OBTENIDAS DE CATEGORIES EN UNA
for index,i in enumerate(df_business.categories.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CATEGORIES'][index]=''.join(arr)

**Creamos la columna 'HOURS' del DataFrame df_business extrayendo los datos y agregando .**

In [ ]:
for index,i in enumerate(df_business.hours.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:
     df_business['HOURS'][index]=arr[0]

In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 42 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null

**Asigno al dataframe  solo a las columnas nuevas**

In [ ]:
df_business=df_business[['BUSINESS_ID','NAME','REVIEW_COUNT','CITY','STATE','ADDRESS','POSTAL_CODE','LATITUDE','LONGITUDE','STARS','IS_OPEN','ATTRIBUTES','CATEGORIES','HOURS']]

In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   BUSINESS_ID   150346 non-null  object
 1   NAME          150346 non-null  object
 2   REVIEW_COUNT  150346 non-null  object
 3   CITY          150346 non-null  object
 4   STATE         150346 non-null  object
 5   ADDRESS       150346 non-null  object
 6   POSTAL_CODE   150346 non-null  object
 7   LATITUDE      150346 non-null  object
 8   LONGITUDE     150346 non-null  object
 9   STARS         150346 non-null  object
 10  IS_OPEN       150346 non-null  object
 11  ATTRIBUTES    136602 non-null  object
 12  CATEGORIES    150346 non-null  object
 13  HOURS         127123 non-null  object
dtypes: object(14)
memory usage: 21.2+ MB


In [ ]:
df_business=df_business.rename(columns={'BUSINESS_ID':'business_id'})
df_business=df_business.rename(columns={'NAME':'name'})
df_business=df_business.rename(columns={'REVIEW_COUNT':'review_count'})
df_business=df_business.rename(columns={'CITY':'city'})
df_business=df_business.rename(columns={'STATE':'state'})
df_business=df_business.rename(columns={'ADDRESS':'address'})
df_business=df_business.rename(columns={'POSTAL_CODE':'postal_code'})
df_business=df_business.rename(columns={'LATITUDE':'lalitude'})
df_business=df_business.rename(columns={'LONGITUDE':'longitude'})
df_business=df_business.rename(columns={'STARS':'stars'})
df_business=df_business.rename(columns={'IS_OPEN':'is_open'})
df_business=df_business.rename(columns={'ATTRIBUTES':'attributes'})
df_business=df_business.rename(columns={'CATEGORIES':'categories'})
df_business=df_business.rename(columns={'HOURS':'hours'})

In [ ]:
df_business.head(3)

,business_id,name,review_count,city,state,address,postal_code,lalitude,longitude,stars,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",7,Santa Barbara,,"1616 Chapala St, Ste 2",93101,34.426679,-119.711197,5.0,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,15,Affton,,87 Grasso Plaza Shopping Center,63123,38.551126,-90.335695,3.0,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,22,Tucson,,5255 E Broadway Blvd,85711,32.223236,-110.880452,3.5,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."


In [ ]:
df_business.state

0           
1           
2           
3         CA
4         MO
          ..
150341    IN
150342    DE
150343    AB
150344    AB
150345    TN
Name: state, Length: 150346, dtype: object

In [ ]:
df_business.to_parquet('BUSINESS_YELP.parquet', index=False, compression='gzip')

In [ ]:
df_business.head(1)

,business_id,name,review_count,city,state,address,postal_code,lalitude,longitude,stars,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",7,Santa Barbara,,"1616 Chapala St, Ste 2",93101,34.426679,-119.711197,5.0,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None


In [ ]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   review_count  150346 non-null  object
 3   city          150346 non-null  object
 4   state         150346 non-null  object
 5   address       150346 non-null  object
 6   postal_code   150346 non-null  object
 7   lalitude      150346 non-null  object
 8   longitude     150346 non-null  object
 9   stars         150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150346 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 21.2+ MB


**Filtramos por el estado de New York**

In [ ]:
df_business[df_business['state']=='NJ'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8535 entries, 45 to 150340
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   8535 non-null   object
 1   name          8535 non-null   object
 2   review_count  8535 non-null   object
 3   city          8535 non-null   object
 4   state         8535 non-null   object
 5   address       8535 non-null   object
 6   postal_code   8535 non-null   object
 7   lalitude      8535 non-null   object
 8   longitude     8535 non-null   object
 9   stars         8535 non-null   object
 10  is_open       8535 non-null   object
 11  attributes    7783 non-null   object
 12  categories    8535 non-null   object
 13  hours         7220 non-null   object
dtypes: object(14)
memory usage: 1000.2+ KB


In [ ]:
df_NYork=df_business[df_business['state']=='NJ'].copy()
df_NYork.head(2)

,business_id,name,review_count,city,state,address,postal_code,lalitude,longitude,stars,is_open,attributes,categories,hours
45,wghnIlMb_i5U46HMBGx9ig,China Dragon Restaurant,23,Tucson,NJ,"1625 W Valencia Rd, Ste 101-103",85746,32.132305,-110.999985,3.0,0,"{'WiFi': ''no'', 'BusinessParking': '{'garage'...","Restaurants, Chinese","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
76,-4dYswJy7SPcbcERvitmIg,Pathmark,34,Philadelphia,NJ,3021 Grays Ferry Ave,19146,39.940403,-75.193297,2.5,0,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Food, Grocery","{'Monday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'W..."


In [ ]:
df_NYork[df_NYork['name']=="McDonald's"].business_id.count()

50

In [ ]:
df_NYork[df_NYork['name']=="McDonald's"].head(3)

,business_id,name,review_count,city,state,address,postal_code,lalitude,longitude,stars,is_open,attributes,categories,hours
361,aNXw3PkXVt8ANwLyCfcmpg,McDonald's,17,Lansdale,NJ,2333 Welsh Rd,19446,40.263706,-75.317916,1.5,1,"{'WiFi': 'u'free'', 'GoodForKids': 'True', 'Bu...","Restaurants, Fast Food, Burgers, Food, Coffee ...","{'Monday': '6:0-23:0', 'Tuesday': '6:0-23:0', ..."
3161,CxoRg-ZcXdpqXWmUJliYog,McDonald's,28,Hudson,NJ,9441 St Rd 52,34669,28.331619,-82.658409,2.5,1,"{'HasTV': 'True', 'Alcohol': 'u'none'', 'Busin...","Restaurants, Food, Fast Food, Coffee & Tea, Bu...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
3593,2gTnSpYBmg_5JubGpCtSQQ,McDonald's,41,Fishers,NJ,9611 N By Ne Blvd,46038,39.927626,-86.025654,2.0,1,"{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...","Coffee & Tea, Food, Burgers, Fast Food, Restau...","{'Monday': '5:0-1:0', 'Tuesday': '5:0-1:0', 'W..."


**Realizamos converciones en los tipos de dato nesesarias**

In [ ]:
#CONVERSIONES A REALIZAR AL FINAL DE EL TRATAMIENTO DE TODOS LOS DATOS
df_NYork["business_id"] = df_NYork["business_id"].astype('str')
df_NYork["name"] = df_NYork["name"].astype('str')
df_NYork["review_count"] = df_NYork["review_count"].astype('int')
df_NYork["city"] = df_NYork["city"].astype('str')
df_NYork["state"] = df_NYork["state"].astype('str')
df_NYork["address"] = df_NYork["address"].astype('str')
df_NYork["postal_code"] = df_NYork["postal_code"].apply(pd.to_numeric, errors='coerce')
df_NYork["stars"] = df_NYork["stars"].apply(pd.to_numeric, errors='coerce')


In [ ]:
df_NYork.to_parquet('BUSINESS_YELP_YORK.parquet', index=False, compression='gzip')

In [ ]:
df_NYork.to_csv("BUSINESS_YELP_YORK.csv")

**Generamos un nuevo dataframe con 'business_id',
'categories'**

In [ ]:
#Tabla de dimension de categorias
df_cat = df_NYork.loc[:, ['business_id','categories']]
df_cat.head(3)

,business_id,categories
45,wghnIlMb_i5U46HMBGx9ig,"Restaurants, Chinese"
76,-4dYswJy7SPcbcERvitmIg,"Food, Grocery"
86,sE6jSnvMts_MAn-b4OkMAw,"Pet Groomers, Pet Sitting, Pets, Pet Services"


In [ ]:
# Dividir los valores de la columna "categories" por coma y expandirlos en filas
df_cat['categories'] = df_cat['categories'].str.split(',')
df_cat = df_cat[['business_id', 'categories']].explode('categories')

In [ ]:
df_cat.head(3)

,business_id,categories
45,wghnIlMb_i5U46HMBGx9ig,Restaurants
45,wghnIlMb_i5U46HMBGx9ig,Chinese
76,-4dYswJy7SPcbcERvitmIg,Food


In [ ]:
df_cat.to_csv("BusinessCategorias_Yelp.csv")

In [ ]:
#Creamos un array con las categorias unicas
categorias = df_cat["categories"].unique()
#Creamos la tabla de dimension de categorias de yelp
df_categorias = pd.DataFrame(categorias, columns=["Descripcion"])
df_categorias['IdCategoria'] = df_categorias.index

In [ ]:
df_categorias.head(5)

,Descripcion,IdCategoria
0,Restaurants,0
1,Chinese,1
2,Food,2
3,Grocery,3
4,Pet Groomers,4


**Carga en la tabla de dimensiones de categorias disponibles de los locales de la franquicia McDonald's en el estado de New York**

In [ ]:
df_categorias.to_csv("Categorias_Yelp.csv")

**Generamos tabla de dimension atributos de locales de franquicia en el estado New York**

In [ ]:
df_atribute = df_NYork.loc[:, ["business_id",'attributes']]
df_atribute.head()

,business_id,attributes
45,wghnIlMb_i5U46HMBGx9ig,"{'WiFi': ''no'', 'BusinessParking': '{'garage'..."
76,-4dYswJy7SPcbcERvitmIg,"{'RestaurantsPriceRange2': '2', 'BusinessParki..."
86,sE6jSnvMts_MAn-b4OkMAw,None
91,nUqrF-h9S7myCcvNDecOvw,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':..."
114,4dVrWc4Nhuw9apfk9BWEcA,"{'RestaurantsTakeOut': 'False', 'BikeParking':..."


**Dividimos los elementos de la columna 'attributes'**

In [ ]:
for index, i in enumerate(df_atribute['attributes']):
    if isinstance(i,str):
       df_atribute['attributes'][index]= i[1:-1].split(',')
df_atribute.head()

,business_id,attributes
45,wghnIlMb_i5U46HMBGx9ig,"{'WiFi': ''no'', 'BusinessParking': '{'garage'..."
76,-4dYswJy7SPcbcERvitmIg,"{'RestaurantsPriceRange2': '2', 'BusinessParki..."
86,sE6jSnvMts_MAn-b4OkMAw,None
91,nUqrF-h9S7myCcvNDecOvw,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':..."
114,4dVrWc4Nhuw9apfk9BWEcA,"{'RestaurantsTakeOut': 'False', 'BikeParking':..."


**Expandimos la columna 'attributes' en filas separadas, manteniendo los valores correspondientes de la columna 'business_id'.**

In [ ]:
df_atribute = df_atribute[['business_id', 'attributes']].explode('attributes')
df_atribute.head()

,business_id,attributes
45,wghnIlMb_i5U46HMBGx9ig,WiFi
45,wghnIlMb_i5U46HMBGx9ig,BusinessParking
45,wghnIlMb_i5U46HMBGx9ig,HasTV
45,wghnIlMb_i5U46HMBGx9ig,Alcohol
45,wghnIlMb_i5U46HMBGx9ig,RestaurantsDelivery


**Se eliminan los corchetes y comillas dobles de la columna 'attributes'**

In [ ]:
df_atribute['attributes'] = df_atribute['attributes'].str.replace('{', '').str.replace('}', '')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('"', '')

<ipython-input-67-467e46ee6ade>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_atribute['attributes'] = df_atribute['attributes'].str.replace('{', '').str.replace('}', '')


In [ ]:
df_atribute.head(5)

,business_id,attributes
45,wghnIlMb_i5U46HMBGx9ig,WiFi
45,wghnIlMb_i5U46HMBGx9ig,BusinessParking
45,wghnIlMb_i5U46HMBGx9ig,HasTV
45,wghnIlMb_i5U46HMBGx9ig,Alcohol
45,wghnIlMb_i5U46HMBGx9ig,RestaurantsDelivery


In [ ]:
df_atribute.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69585 entries, 45 to 150340
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  69585 non-null  object
 1   attributes   68833 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


**Creacion de tabla con business_id mas lista de atributos por negocio en el estado de New York**

In [ ]:
df_atribute.to_csv("BusinessAtributos_Yelp.csv")

**Creacion de DataFrame que contiene los valores únicos de la columna 'attributes', junto con una columna de identificación.**

In [ ]:
atributos = df_atribute["attributes"].unique()
df_atributos = pd.DataFrame(atributos, columns=["Descripcion"])
df_atributos['IdAtributos'] = df_atributos.index

In [ ]:
df_atributos.head(3)

,Descripcion,IdAtributos
0,WiFi,0
1,BusinessParking,1
2,HasTV,2


In [ ]:
df_atributos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Descripcion  38 non-null     object
 1   IdAtributos  39 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 752.0+ bytes


**Carga de tabla de atributos unicos en los locales listados en el estado de New York**

In [ ]:
df_atributos.to_csv("Atributos_Yelp.csv")

**Creamos la tabla de dimensiones "Hours" a partir de df_NewYork**

In [ ]:
df_hours = df_NYork.loc[:, ["business_id",'hours']]

In [ ]:
df_hours.head(3)

,business_id,hours
45,wghnIlMb_i5U46HMBGx9ig,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
76,-4dYswJy7SPcbcERvitmIg,"{'Monday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'W..."
86,sE6jSnvMts_MAn-b4OkMAw,"{'Monday': '8:0-16:0', 'Tuesday': '8:0-16:0', ..."


In [ ]:
df_hours.hours

45        {'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...
76        {'Monday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'W...
86        {'Monday': '8:0-16:0', 'Tuesday': '8:0-16:0', ...
91        {'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', '...
114       {'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...
                                ...                        
150285    {'Monday': '8:30-17:0', 'Tuesday': '8:30-17:0'...
150302    {'Monday': '0:0-0:0', 'Tuesday': '6:30-20:0', ...
150311    {'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...
150317    {'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...
150340    {'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...
Name: hours, Length: 8535, dtype: object

**Creamos el dataframe df_Hour_detalle**

In [ ]:
df_hour_detalle=df_hours[['business_id','hours']]

In [ ]:
df_hour_detalle

,business_id,hours
45,wghnIlMb_i5U46HMBGx9ig,Monday
45,wghnIlMb_i5U46HMBGx9ig,Tuesday
45,wghnIlMb_i5U46HMBGx9ig,Wednesday
45,wghnIlMb_i5U46HMBGx9ig,Thursday
45,wghnIlMb_i5U46HMBGx9ig,Friday
...,...,...
150340,hn9Toz3s-Ei3uZPt7esExA,Wednesday
150340,hn9Toz3s-Ei3uZPt7esExA,Thursday
150340,hn9Toz3s-Ei3uZPt7esExA,Friday
150340,hn9Toz3s-Ei3uZPt7esExA,Saturday


In [ ]:
df_hours.to_csv("BusinessHoras.csv")

**Desglosamos los diferentes horarios en diferentes filas para cada valor de forma individual**

In [ ]:
df_hour_detalle=df_hour_detalle[['business_id','hours']].explode('hours')
df_hour_detalle.head()

,business_id,hours
45,wghnIlMb_i5U46HMBGx9ig,Monday
45,wghnIlMb_i5U46HMBGx9ig,Tuesday
45,wghnIlMb_i5U46HMBGx9ig,Wednesday
45,wghnIlMb_i5U46HMBGx9ig,Thursday
45,wghnIlMb_i5U46HMBGx9ig,Friday


**Creamos un dataframe llamado hour con las horas unicas de df_hour_detalle**

In [ ]:
hours=df_hour_detalle['hours'].unique()
hours

array(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday', None], dtype=object)

**creacion de  DataFrame que contiene  "Descripcion" (valores de la columna 'hours' )**

In [ ]:
DescHour=pd.DataFrame(hours,columns=["Descripcion"])
DescHour['id_hour']=DescHour.index
DescHour

,Descripcion,id_hour
0,Monday,0
1,Tuesday,1
2,Wednesday,2
3,Thursday,3
4,Friday,4
5,Saturday,5
6,Sunday,6
7,None,7


**Cargamos en tabla de dimension Horarios**

In [ ]:
DescHour.to_csv('Horarios_Yelp.csv',index=False)

**Hacemos una union entre df_hour_detalle y DescHour**

In [ ]:
df_hour_detalle.merge(DescHour,left_on='hours',right_on='Descripcion',how='inner')[['business_id','id_hour']]


,business_id,id_hour
0,wghnIlMb_i5U46HMBGx9ig,0
1,-4dYswJy7SPcbcERvitmIg,0
2,sE6jSnvMts_MAn-b4OkMAw,0
3,nUqrF-h9S7myCcvNDecOvw,0
4,4dVrWc4Nhuw9apfk9BWEcA,0
...,...,...
46822,76vbrj2OZhgPJUv46Doe5Q,7
46823,yDPwHWr9bY88DhoQmMPWmg,7
46824,c6RWw740cIB9-efs9tSLyg,7
46825,oAsfYTJE3cOJXk7Dmb-1TQ,7


**Cargamos en tabla de dimension Horarios por Negocio**

In [ ]:
df_hour_detalle.to_csv('Business_Horarios_Yelp.csv',index=False)

**Creamos el dataframe df_detalleatributo a partir de la union entre df_atribute y df_atributos**

In [ ]:
df_detalleatributo = df_atribute.merge(df_atributos, left_on="attributes", right_on="Descripcion", how="inner")

In [ ]:
df_detalleatributo.head(5)

,business_id,attributes,Descripcion,IdAtributos
0,wghnIlMb_i5U46HMBGx9ig,WiFi,WiFi,0
1,nUqrF-h9S7myCcvNDecOvw,WiFi,WiFi,0
2,9C2rpb56aQvW0ViZHK9sPw,WiFi,WiFi,0
3,2oxrOO3c9_mQmqM9kwEm9Q,WiFi,WiFi,0
4,QZU7TcrztBb3tXaPbVCkXg,WiFi,WiFi,0


**Eliminamos las columnas "attributes" y "Descripcion" de df_detalleatributo**

In [ ]:
df_detalleatributo.drop(columns = "attributes",inplace=True)
df_detalleatributo.drop(columns = "Descripcion",inplace=True)

In [ ]:
df_detalleatributo.tail(5)

,business_id,IdAtributos
69580,ccNH_y1-c58tX74o4V4mlw,37
69581,bNSBYd-wASjWZ3hZWb0sGQ,38
69582,lvi4u_XVzvL06EAj0hUmfw,38
69583,SvA2IQ9SyI1Dh7dtwNNs3A,38
69584,K2MrnWrEEg9FzHrm9lve8A,38


**Carga en tabla detalles de Atributo por Negocio**

In [ ]:
df_detalleatributo.to_csv("Business_Atributos_Yelp.csv")

## **tip.json**
---

**Abrimos el archivo json**

In [ ]:
df_tip=pd.read_json(r'/content/drive/MyDrive/P_F_H/tip.json',lines=True)
df_tip.head(6)
# archivo a entrecruzar por id de usuario

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
5,trf3Qcz8qvCDKXiTgjUcEg,7Rm9Ba50bw23KTA8RedZYg,"Chili Cup + Single Cheeseburger with onion, pi...",2012-03-13 04:00:52,0


In [ ]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


 **Obtenemos un DataFrame que contenga solo las filas correspondientes a los negocios presentes en BusinessYelp**

In [ ]:
df_tip=df_tip[df_tip['business_id'].isin(df_NYork.business_id.unique().tolist())]

In [ ]:
df_tip2=df_NYork[df_NYork['business_id'].isin(df_tip.business_id.unique().tolist())]
df_tip2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6037 entries, 76 to 150340
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   6037 non-null   object 
 1   name          6037 non-null   object 
 2   review_count  6037 non-null   int64  
 3   city          6037 non-null   object 
 4   state         6037 non-null   object 
 5   address       6037 non-null   object 
 6   postal_code   5801 non-null   float64
 7   lalitude      6037 non-null   object 
 8   longitude     6037 non-null   object 
 9   stars         6037 non-null   float64
 10  is_open       6037 non-null   object 
 11  attributes    5711 non-null   object 
 12  categories    6037 non-null   object 
 13  hours         5186 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 707.5+ KB


In [ ]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52302 entries, 0 to 52301
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           52302 non-null  object        
 1   business_id       52302 non-null  object        
 2   text              52302 non-null  object        
 3   date              52302 non-null  datetime64[ns]
 4   compliment_count  52302 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.4+ MB


**Hacemos una unión df_tip con df_NYork**

In [ ]:
df_tip=df_NYork.merge(df_tip,left_on='business_id',right_on='business_id',how='right')[['user_id', 'business_id', 'text', 'date', 'compliment_count']]

In [ ]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52302 entries, 0 to 52301
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           52302 non-null  object        
 1   business_id       52302 non-null  object        
 2   text              52302 non-null  object        
 3   date              52302 non-null  datetime64[ns]
 4   compliment_count  52302 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.4+ MB


**Hacemos una unión df_tip con UsuarioYelp**

In [ ]:
df_User_Yelp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987897 entries, 0 to 1987896
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        object        
 1   review_count   int64         
 2   yelping_since  datetime64[ns]
 3   average_stars  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 75.8+ MB


In [ ]:
df_tip3=df_User_Yelp.merge(df_tip,left_on='user_id',right_on='user_id',how='inner')[['user_id', 'text', 'date', 'compliment_count','review_count','yelping_since','average_stars']]
df_tip3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52302 entries, 0 to 52301
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           52302 non-null  object        
 1   text              52302 non-null  object        
 2   date              52302 non-null  datetime64[ns]
 3   compliment_count  52302 non-null  int64         
 4   review_count      52302 non-null  int64         
 5   yelping_since     52302 non-null  datetime64[ns]
 6   average_stars     52302 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 3.2+ MB


**Cargamos datos en tabla contiene informacion de los tip de usuarios ,de la franquicia McDonald's en el estado New York**

In [ ]:
df_tip3.to_csv('BusinessTipsYelp.csv',index=False)

## **review.json**
---

**Leemos el archivo review.json de Yelp**

In [ ]:
df = pd.read_json("/content/drive/MyDrive/P_F_H/review.json", orient ='index')
print(df)

FileNotFoundError: ignored

**Eliminamos duplicados y reindexamos**

In [ ]:
df_review= df_review.drop_duplicates(subset=["review_id", "text"], keep="first")
df_review['id_review'] = df_review.index

**Creamos un dataframe con "id_review", "review_id" llamado df_ReviewYelp**

In [ ]:
df_ReviewYelp = df_review[["id_review", "review_id"]]

**Exportamos el dataframe ReviewYelpId**

In [ ]:
df_ReviewYelp.to_csv("Datasets/ReviewYelpId.csv")

**Eliminamos columna review_id de df_Review**

In [ ]:
df_review.drop(columns = "review_id",inplace=True)

**Unimos el dataframe df_review con dfusuario**

In [ ]:
df_review = df_review.merge(dfusuario, left_on="user_id", right_on="yelp_id", how="inner")

**Creamos el dataframe df_reviewfinal a partir de df_review y BusinessYelp**

In [ ]:
df_reviewfinal = df_review.merge(BusinessYelp, left_on="business_id", right_on="businessYelp_id", how="inner")

**Se eliminan las columnas "user_id" y "yelp_id" del dataframe df_reviewfinal**

In [ ]:
df_reviewfinal.drop(columns = "business_id",inplace=True)
df_reviewfinal.drop(columns = "businessYelp_id",inplace=True)
df_reviewfinal.drop(columns = "Yelp_id",inplace=True)

**Exportamos dataframe review**

In [ ]:
df_reviewfinal.to_csv("Review.csv")

# **ETL_Archivos de Google Maps**
---

## **Review Estados**
---

**Cargamos todas las reviews correspondientes al estado de New York**

In [6]:
#Cargamos todas las reviews correspondientes al estado de New York
df_rev2=pd.read_json(r'/content/drive/MyDrive/P_F_H/2.json',lines=True)
df_rev3=pd.read_json(r'/content/drive/MyDrive/P_F_H/3.json',lines=True)
df_rev4=pd.read_json(r'/content/drive/MyDrive/P_F_H/4.json',lines=True)
df_rev5=pd.read_json(r'/content/drive/MyDrive/P_F_H/5.json',lines=True)
df_rev6=pd.read_json(r'/content/drive/MyDrive/P_F_H/6.json',lines=True)
df_rev7=pd.read_json(r'/content/drive/MyDrive/P_F_H/7.json',lines=True)
df_rev8=pd.read_json(r'/content/drive/MyDrive/P_F_H/8.json',lines=True)
df_rev9=pd.read_json(r'/content/drive/MyDrive/P_F_H/9.json',lines=True)
df_rev10=pd.read_json(r'/content/drive/MyDrive/P_F_H/10.json',lines=True)
df_rev11=pd.read_json(r'/content/drive/MyDrive/P_F_H/11.json',lines=True)
df_rev12=pd.read_json(r'/content/drive/MyDrive/P_F_H/12.json',lines=True)
df_rev13=pd.read_json(r'/content/drive/MyDrive/P_F_H/13.json',lines=True)
df_rev14=pd.read_json(r'/content/drive/MyDrive/P_F_H/14.json',lines=True)
df_rev15=pd.read_json(r'/content/drive/MyDrive/P_F_H/15.json',lines=True)
df_rev16=pd.read_json(r'/content/drive/MyDrive/P_F_H/16.json',lines=True)
df_rev17=pd.read_json(r'/content/drive/MyDrive/P_F_H/17.json',lines=True)
df_rev18=pd.read_json(r'/content/drive/MyDrive/P_F_H/18.json',lines=True)


**Se unen todos los archivos de las reviews en uno**

In [7]:
data_rev_NY=pd.concat([df_rev2,df_rev3,df_rev4,df_rev5,df_rev6,df_rev7,df_rev8,df_rev9,df_rev10,df_rev11,df_rev12,df_rev13,df_rev14,df_rev15,df_rev16,df_rev17,df_rev18],sort=False,ignore_index=True)
data_rev_NY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550000 entries, 0 to 2549999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 155.6+ MB


**Eliminamos columnas que no se usarán en el proyecto**

In [8]:
# Dropeamos las columnas que no usaremos
data_rev_NY.drop(["name", "pics"], axis=1, inplace=True)

**Cambios en los formato de los datos**

In [9]:
# Convertir a formatos nesesarios
data_rev_NY['time'] = pd.to_datetime(data_rev_NY['time'], format='%Y-%m-%d')
data_rev_NY["text"]=data_rev_NY["text"].astype('string')
data_rev_NY['gmap_id']=data_rev_NY['gmap_id'].astype('string')
data_rev_NY["resp"]=data_rev_NY["resp"].astype('string')

In [10]:
data_rev_NY[data_rev_NY['gmap_id']=="0x80904101ce001281:0x76db23c5d22346ae"]

,user_id,time,rating,text,resp,gmap_id


In [11]:
data_rev_NY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550000 entries, 0 to 2549999
Data columns (total 6 columns):
 #   Column   Dtype         
---  ------   -----         
 0   user_id  float64       
 1   time     datetime64[ns]
 2   rating   int64         
 3   text     string        
 4   resp     string        
 5   gmap_id  string        
dtypes: datetime64[ns](1), float64(1), int64(1), string(3)
memory usage: 116.7 MB


**Creamos la tabla de Datos de reviews de estado de New York**

In [12]:
data_rev_NY.to_parquet('Reviews_Gmaps_NY', index=False, compression='gzip')

ArrowInvalid: ignored

In [ ]:
data_rev_NY.to_csv('Reviews_Gmaps_NY',escapechar='\\',index=False)

## **Carpeta Metadata_Sitios**
---

In [13]:
#Cargamos el archivo correspondiente a los datos del estado de New York
df_data_locales_NJ=pd.read_json(r'/content/drive/MyDrive/P_F_H/1.json',lines=True)
df_data_locales_NJ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              274994 non-null  object 
 1   address           264939 non-null  object 
 2   gmap_id           275001 non-null  object 
 3   description       13155 non-null   object 
 4   latitude          275001 non-null  float64
 5   longitude         275001 non-null  float64
 6   category          272740 non-null  object 
 7   avg_rating        275001 non-null  float64
 8   num_of_reviews    275001 non-null  int64  
 9   price             13450 non-null   object 
 10  hours             192448 non-null  object 
 11  MISC              194972 non-null  object 
 12  state             195523 non-null  object 
 13  relative_results  238771 non-null  object 
 14  url               275001 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 31.5+ MB


In [14]:
df_data_locales_NJ.head(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [15]:
df_data_locales_NJ['name']=df_data_locales_NJ['name'].astype('string')
df_data_locales_NJ['address']=df_data_locales_NJ['address'].astype('string')
df_data_locales_NJ['gmap_id']=df_data_locales_NJ['gmap_id'].astype('string')
df_data_locales_NJ['description']=df_data_locales_NJ['description'].astype('string')

**Crea un subconjunto de todos los locales,obteniendo solo los pertenecientes a la franquicia de New York**

In [16]:
df_data_locales_NJ[df_data_locales_NJ["name"]=="McDonald's"].name.count()

29

In [17]:
df_NY_Locales=df_data_locales_NJ[df_data_locales_NJ["name"]=="McDonald's"].copy()
df_NY_Locales.head(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
5012,McDonald's,"McDonald's, 1205 S Main St, Manteca, CA 95336",0x80904101ce001281:0x76db23c5d22346ae,"Classic, long-running fast-food chain known fo...",37.785995,-121.218062,"[Fast food restaurant, Breakfast restaurant, C...",2.4,48,$,None,"{'Service options': ['Takeout', 'Delivery'], '...",None,"[0x8090410018dc2657:0xed7a807ae3eeee6d, 0x8090...",https://www.google.com/maps/place//data=!4m2!3...
6679,McDonald's,"McDonald's, 1000 Palisades Center Dr, West Nya...",0x89c2e9cf8e139235:0x24bfb20e9e09f260,"Classic, long-running fast-food chain known fo...",41.097768,-73.955392,"[Fast food restaurant, Breakfast restaurant, C...",2.2,18,$,None,"{'Service options': ['Delivery'], 'Highlights'...",None,"[0x89c2e9e6ef010ddb:0xe923f7207b70d6f9, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...


In [18]:
df_NY_Locales.loc[5012].MISC.items()

dict_items([('Service options', ['Takeout', 'Delivery']), ('Accessibility', ['Wheelchair accessible entrance', 'Wheelchair accessible parking lot', 'Wheelchair accessible restroom', 'Wheelchair accessible seating']), ('Offerings', ['All you can eat', 'Coffee', "Kids' menu", 'Late-night food', 'Vegetarian options']), ('Dining options', ['Breakfast', 'Lunch', 'Dinner', 'Catering', 'Dessert', 'Seating']), ('Amenities', ['Gender-neutral restroom', 'Good for kids', 'High chairs', 'Restroom']), ('Payments', ['Cash-only', 'NFC mobile payments', 'Credit cards'])])

In [19]:
df_atribute_gmaps = df_NY_Locales.loc[:, ["gmap_id",'MISC']]
df_atribute_gmaps.head()

,gmap_id,MISC
5012,0x80904101ce001281:0x76db23c5d22346ae,"{'Service options': ['Takeout', 'Delivery'], '..."
6679,0x89c2e9cf8e139235:0x24bfb20e9e09f260,"{'Service options': ['Delivery'], 'Highlights'..."
6690,0x89c259a9b55adb77:0xfe5e87207e736efc,{'Service options': ['Delivery']}
7564,0x864c2b8770fc957d:0xdbc6f271dec0dcef,"{'Service options': ['Curbside pickup', 'Takeo..."
16258,0x86de3d67b2b54843:0xa54f3893ef44d96,"{'Service options': ['Delivery'], 'Dining opti..."


In [116]:
df_atribute_gmaps['gmap_id'][5012]

'0x80904101ce001281:0x76db23c5d22346ae'

In [141]:
fil=pd.DataFrame()
filtotal=pd.DataFrame()
k=0
for j in df_atribute_gmaps['MISC'].index:

    for index, i in enumerate(df_atribute_gmaps['MISC']):
       # k=k+1

        for clave in i:
            fil[clave]=pd.DataFrame(i.get(clave))
        fil['gmap_id']=df_atribute_gmaps['gmap_id'][j]
      # fil['count']=k
        filtotal=pd.concat([filtotal,fil])

filtotal


,Service options,Accessibility,Offerings,Dining options,Amenities,Payments,gmap_id,Highlights,Popular for,Atmosphere,Crowd
0,Takeout,Wheelchair accessible entrance,All you can eat,Breakfast,Gender-neutral restroom,Cash-only,0x80904101ce001281:0x76db23c5d22346ae,NaN,NaN,NaN,NaN
1,Delivery,Wheelchair accessible parking lot,Coffee,Lunch,Good for kids,NFC mobile payments,0x80904101ce001281:0x76db23c5d22346ae,NaN,NaN,NaN,NaN
0,Delivery,Wheelchair accessible restroom,Alcohol,Breakfast,Good for kids,NFC mobile payments,0x80904101ce001281:0x76db23c5d22346ae,Fast service,Breakfast,Casual,Groups
1,NaN,NaN,Kids' menu,Lunch,High chairs,NaN,0x80904101ce001281:0x76db23c5d22346ae,Great coffee,Lunch,NaN,NaN
0,Delivery,Wheelchair accessible restroom,Alcohol,Breakfast,Good for kids,NFC mobile payments,0x80904101ce001281:0x76db23c5d22346ae,Fast service,Breakfast,Casual,Groups
...,...,...,...,...,...,...,...,...,...,...,...
1,Delivery,NaN,Halal food,Lunch,Good for kids,NFC mobile payments,0x80c8c5cd970109b1:0x93ff917d0d1de763,Great coffee,Lunch,NaN,NaN
0,Drive-through,Wheelchair accessible entrance,Coffee,Breakfast,Good for kids,Debit cards,0x80c8c5cd970109b1:0x93ff917d0d1de763,Fast service,Solo dining,Casual,Groups
1,Delivery,NaN,Kids' menu,Lunch,Restroom,NaN,0x80c8c5cd970109b1:0x93ff917d0d1de763,NaN,NaN,NaN,NaN
0,Delivery,Wheelchair accessible entrance,Coffee,Breakfast,Good for kids,Debit cards,0x80c8c5cd970109b1:0x93ff917d0d1de763,Fast service,Solo dining,Casual,Groups


In [143]:
for j , k in enumerate(filtotal['gmap_id'].unique()):
    print(j,k)

0 0x80904101ce001281:0x76db23c5d22346ae
1 0x89c2e9cf8e139235:0x24bfb20e9e09f260
2 0x89c259a9b55adb77:0xfe5e87207e736efc
3 0x864c2b8770fc957d:0xdbc6f271dec0dcef
4 0x86de3d67b2b54843:0xa54f3893ef44d96
5 0x8833e48bd320d61d:0xa1f87acf7960fc84
6 0x89b62b830b97b9a9:0xaeb64e76075a65d2
7 0x87134d26adc91615:0x7c5d16cf3530db8a
8 0x872baf95fc7f1e9d:0x4aad787399f2dcb6
9 0x89b5eee935b6f9a9:0x948fec1f53c3a40f
10 0x8824d24252905655:0xa49d45451a7cdfbf
11 0x7952317f2e3ac36f:0x3b7302c9d9e737d8
12 0x87d9290e7f56bbf1:0x927d4d189b887ba4
13 0x880511dec387b13d:0x34238816bf39ebe2
14 0x87221d869f8613c3:0x22cedc093e5861e3
15 0x87c0e659ed9de565:0x748be909e28facd2
16 0x880966b1b0d42a35:0x59243c5f307e9807
17 0x80dd26e819f827c3:0x55d8787bb70a2ce8
18 0x89b9df4942bf613d:0xea88a1a371741509
19 0x79523116da789ced:0xea1820a40a0cba6b
20 0x52cf95d9b0842137:0x3789a0cccb8dee0f
21 0x8868e886b0dd5423:0x58236fd6c2eada13
22 0x87b442e37fe4d597:0x95c570ed79c672df
23 0x80c8c5cd970109b1:0x93ff917d0d1de763


In [ ]:
df_atribute_gmaps.loc[5012].MISC

5012    Service options
5012      Accessibility
5012          Offerings
5012     Dining options
5012          Amenities
5012           Payments
Name: MISC, dtype: object

**Cargamos los metadatos del estado de New York**

In [ ]:
#Cargamos el archivo correspondiente a los datos del estado de New York
df_meta_locales_NJ=pd.read_json(r'/content/drive/MyDrive/P_F_H/meta_goo/1.json',lines=True)
df_meta_locales_NJ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              274994 non-null  object 
 1   address           264939 non-null  object 
 2   gmap_id           275001 non-null  object 
 3   description       13155 non-null   object 
 4   latitude          275001 non-null  float64
 5   longitude         275001 non-null  float64
 6   category          272740 non-null  object 
 7   avg_rating        275001 non-null  float64
 8   num_of_reviews    275001 non-null  int64  
 9   price             13450 non-null   object 
 10  hours             192448 non-null  object 
 11  MISC              194972 non-null  object 
 12  state             195523 non-null  object 
 13  relative_results  238771 non-null  object 
 14  url               275001 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 31.5+ MB


**Realizamos cambios en lo tipos de datos nesesarios**

In [ ]:
df_meta_locales_NJ['name']=df_meta_locales_NJ['name'].astype('string')
df_meta_locales_NJ['address']=df_meta_locales_NJ['address'].astype('string')
df_meta_locales_NJ['gmap_id']=df_meta_locales_NJ['gmap_id'].astype('string')
df_meta_locales_NJ['description']=df_meta_locales_NJ['description'].astype('string')

In [ ]:
df_meta_locales_NJ[df_meta_locales_NJ["name"]=="McDonald's"].gmap_id.head(2)

5012    0x80904101ce001281:0x76db23c5d22346ae
6679    0x89c2e9cf8e139235:0x24bfb20e9e09f260
Name: gmap_id, dtype: string

**Realizamos  entrecruzamiento de datos de Metadata_Sitios con el estado de New york**



In [ ]:

# Filtrar los registros de Metadata que tienen el mismo 'gmap_id' que los de New York
Meta_Gmaps_NY= df_meta_locales_NJ [df_meta_locales_NJ ['gmap_id'].isin(data_rev_NY ['gmap_id'])].copy()

# Agregar la nueva columna 'New York' al nuevo DataFrame
Meta_Gmaps_NY['State'] = 'NJ'

In [ ]:
McD_Meta_NY=Meta_Gmaps_NY[Meta_Gmaps_NY["name"]=="McDonald's"].copy()
McD_Meta_NY.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 5012 to 263786
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              29 non-null     object 
 1   address           29 non-null     object 
 2   gmap_id           29 non-null     string 
 3   description       24 non-null     object 
 4   latitude          29 non-null     float64
 5   longitude         29 non-null     float64
 6   category          29 non-null     object 
 7   avg_rating        29 non-null     float64
 8   num_of_reviews    29 non-null     int64  
 9   price             22 non-null     object 
 10  hours             13 non-null     object 
 11  MISC              29 non-null     object 
 12  state             13 non-null     object 
 13  relative_results  17 non-null     object 
 14  url               29 non-null     object 
 15  State             29 non-null     object 
dtypes: float64(3), int64(1), object(11), st

In [ ]:
Meta_Reviews_NY=pd.merge(df_data_locales_NJ, data_rev_NY, on='gmap_id')
Meta_Reviews_NY.info()

In [ ]:
Meta_Locales_Reviews_NY=pd.merge(df_data_locales_NJ, data_rev_NY, left_on='gmap_id', right_on='gmap_id',how='inner')

**Se eliminan las columnas que no se usaran**

In [ ]:

Meta_Gmaps_NY.drop(["url", "relative_results"], axis=1, inplace=True)
Meta_Gmaps_NY.drop(['description', 'price'], axis=1, inplace=True)

In [ ]:
Meta_Gmaps_NY.info()

NameError: ignored


**Cargamos  la tabla  con los datos dde usuario por negocio**

In [ ]:
# Exportar el DataFrame
Meta_Gmaps_NY.to_csv('Business_Reviews_Gmaps.csv', escapechar='\\', index=False)